## Note: The examples used in this notebook are meant to demonstrate functionality, not showcase estimation accuracy. The synthetic data generated for these examples may not reflect real-world scenarios accurately. The purpose is to illustrate how to use the `estimate` function with different settings and validation methods.
### Note: This notebook was run on my local machine and the results are displayed here. The code cannot be run in Colab. The execution times displayed were obtained on a All results were obtained on a 2021 10-core Apple M1 Pro CPU.

In [1]:
import jax.numpy as jnp
from src.mcnnm.utils import generate_data
from src.mcnnm.wrappers import complete_matrix, estimate

## Example 1: Staggered Adoption with Cross-Validation (Default)
In this example, we generate a dataset with covariates and staggered adoption treatment assignment and use the default cross-validation method for selecting the regularization parameters. Cross-validation is currently not parallelized and may take longer to run on large datasets or smaller processors.

In [2]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=True,
    Z_cov=True,
    V_cov=True,
    seed=2024,
    noise_scale=0.1,
    autocorrelation=0.0,
    assignment_mechanism="staggered",
    treatment_probability=0.1,
)

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="cv",
    K=3,
    n_lambda=30,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.3f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 5.435
Chosen lambda_L: 0.0000, lambda_H: 0.1209


The `generate_data` function is used to create a synthetic dataset with staggered adoption treatment assignment. The assignment_mechanism parameter is set to `staggered`, which means that each unit adopts the treatment at a random time point with a specified probability.
By default, the estimate function uses cross-validation to select the optimal regularization parameters lambda_L and lambda_H. Cross-validation splits the data into K folds (default is 5) and evaluates the model performance on each fold to select the best parameters.

## Example 2: Block Assignment with Holdout Validation
In this example, we generate a dataset without covariates using block treatment assignment and use holdout validation for selecting the regularization parameters.

In [3]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=False,
    Z_cov=False,
    V_cov=False,
    seed=2024,
    noise_scale=0.1,
    autocorrelation=0.0,
    assignment_mechanism="block",
    treated_fraction=0.1,
)

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="cv",
    K=2,
    n_lambda=10,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.3f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 5.657
Chosen lambda_L: 0.0000, lambda_H: 0.0342


Here, the `assignment_mechanism` is set to `block`, which means that a specified fraction of units (determined by `treated_fraction`) are treated in the second half of the time periods.
The validation_method parameter in the estimate function is set to `holdout`, indicating that holdout validation should be used for selecting the regularization parameters. Holdout validation splits the data into a training set and a validation set based on time. It uses the earlier time periods for training and the later time periods for validation. Holdout validation is typically faster than cross-validation but may be less accurate, especially if the number of time periods is small.

## Example 3: Single Treated Unit with Covariates
In this example, we generate a dataset with a single treated unit and include covariates in the estimation.

In [4]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=True,
    Z_cov=True,
    V_cov=True,
    seed=2024,
    noise_scale=0.1,
    assignment_mechanism="single_treated_unit",
)

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="cv",
    K=3,
    n_lambda=20,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.3f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 6.856
Chosen lambda_L: 0.0000, lambda_H: 0.0981


The `assignment_mechanism` is set to `'single_treated_unit'`, which means that only one randomly selected unit is treated in the second half of the time periods.

In this example, we include unit-specific covariates `X`, time-specific covariates `Z`, and unit-time specific covariates `V` in the estimation. The `estimate` function automatically handles the presence of covariates and estimates their coefficients along with the treatment effect.

With this specific dataset, the estimated treatment effect is not close to the true treatment effect, as the single treated unit leads to the cross-validation method struggling to find a valid loss during the parameter selection process. The warning message "No valid loss found in cross_validate" indicates that the cross-validation procedure could not find a suitable set of regularization parameters that yielded a finite loss value.

This issue arises because with only a single treated unit, there might not be enough information to reliably estimate the treatment effect, especially when using cross-validation. The limited treatment variation can make it challenging for the model to distinguish the treatment effect from the noise in the data.

In such cases, it may be more appropriate to use a different validation method, such as holdout validation, or to rely on domain knowledge to set the regularization parameters manually. Additionally, increasing the number of observations or treated units can help improve the estimation accuracy and stability.

It's important to note that the performance of the estimation method can be sensitive to the specific dataset and the chosen assignment mechanism. While the `estimate` function aims to handle various scenarios, there may be limitations in extreme cases like having only a single treated unit. It's always a good practice to carefully evaluate the results, consider the characteristics of the dataset, and interpret the findings in the context of the specific application.

## Example 4: Estimation with Holdout Validation
In this example, we generate data without covariates and staggered adoption treatment assignment and use holdout validation for selecting the regularization parameters. The `max_window_size` parameter is set to 80, which controls the maximum size of the training window in the holdout validation process.

In [5]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=100,
    unit_fe=True,
    time_fe=True,
    X_cov=False,
    Z_cov=False,
    V_cov=False,
    seed=2024,
    noise_scale=0.1,
    assignment_mechanism="staggered",
    treatment_probability=0.1,
)

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="holdout",
    initial_window=50,
    max_window_size=80,
    step_size=10,
    horizon=5,
    K=5,
    n_lambda=30,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.3f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 5.185
Chosen lambda_L: 0.0000, lambda_H: 0.0219


## Example 5: Estimation with Pre-specified Lambda Values

This example shows how to estimate the model using pre-specified lambda values. The data is generated as in Example 1. Specifying both lambda values bypasses the validation process.

In [6]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=True,
    Z_cov=True,
    V_cov=True,
    seed=2024,
    noise_scale=0.1,
    autocorrelation=0.0,
    assignment_mechanism="staggered",
    treatment_probability=0.1,
)

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=0.001,
    lambda_H=0.01,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.2f}")


True effect: 5.0, Estimated effect: 5.57


## Example 6: Estimation with Autocorrelation

In this example, we generate data with autocorrelation (see section 8.3 of [Athey et al. (2021)](https://www.tandfonline.com/doi/full/10.1080/01621459.2021.1891924)) and use a custom Omega matrix in estimation.

In [7]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=True,
    Z_cov=True,
    V_cov=True,
    seed=2024,
    noise_scale=0.1,
    autocorrelation=0.5,
    assignment_mechanism="last_periods",
    last_treated_periods=5,
)

# Create custom Omega matrix with AR(1) structure
rho = 0.5
T = Y.shape[1]
Omega = jnp.power(rho, jnp.abs(jnp.arange(T)[:, None] - jnp.arange(T)))

# Run estimation
results = estimate(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=Omega,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="holdout",
    initial_window=2,
    max_window_size=None,
    step_size=1,
    horizon=1,
    K=3,
    n_lambda=30,
)

print(f"\nTrue effect: {true_params['treatment_effect']}, Estimated effect: {results.tau:.12f}")
print(f"Chosen lambda_L: {results.lambda_L:.4f}, lambda_H: {results.lambda_H:.4f}")


True effect: 5.0, Estimated effect: 4.593369756478
Chosen lambda_L: 0.0000, lambda_H: 0.1695


## Example 7: Matrix Completion

This example demonstrates how to use the `complete_matrix()` function to impute missing values. The data and estimation parameters are identical to Example 2. Note that the `complete_matrix()` function does not estimate the treatment effect but focuses on imputing the missing entries in the outcome matrix. It returns the imputed matrix along with the chosen regularization parameters in an unnamed tuple.

In [8]:
Y, W, X, Z, V, true_params = generate_data(
    nobs=50,
    nperiods=10,
    unit_fe=True,
    time_fe=True,
    X_cov=False,
    Z_cov=False,
    V_cov=False,
    seed=2024,
    noise_scale=0.1,
    autocorrelation=0.0,
    assignment_mechanism="block",
    treated_fraction=0.1,
)

# Run estimation
Y_completed, opt_lambda_L, opt_lambda_H = complete_matrix(
    Y=Y,
    Mask=W,
    X=X,
    Z=Z,
    V=V,
    Omega=None,
    use_unit_fe=True,
    use_time_fe=True,
    lambda_L=None,
    lambda_H=None,
    validation_method="cv",
    K=2,
    n_lambda=10,
)

print(f"Chosen lambda_L: {opt_lambda_L:.4f}, lambda_H: {opt_lambda_H:.4f}")
print(f"Mean absolute error of imputation: {jnp.mean(jnp.abs(Y - Y_completed)):.4f}")
print(f"Mean squared error of imputation: {jnp.mean(jnp.square(Y - Y_completed)):.4f}")
print(f"Mean of Y: {jnp.mean(Y):.4f}, Mean of Y_completed: {jnp.mean(Y_completed):.4f}")

Chosen lambda_L: 0.0000, lambda_H: 0.0342
Mean absolute error of imputation: 0.3960
Mean squared error of imputation: 2.6292
Mean of Y: 10.2857, Mean of Y_completed: 9.8897


# Covariates
The `generate_data` function allows you to include three types of covariates in the generated dataset:

1. **Unit-specific covariates (X):** These are characteristics or features that vary across units but remain constant over time. For example, in a study of students' academic performance, unit-specific covariates could include variables like gender, age, or socioeconomic status. These covariates capture the inherent differences between units that may influence the outcome variable.
2. **Time-specific covariates (Z):** These are factors that change over time but are the same for all units at each time point. For instance, in an analysis of sales data, time-specific covariates could include variables like market trends, seasonal effects, or economic indicators. These covariates reflect the temporal variations that affect all units simultaneously.
3. **Unit-time specific covariates (V):** These are covariates that vary both across units and over time. They capture the unique characteristics of each unit at each time point. For example, in a healthcare study, unit-time specific covariates could include individual patients' medical measurements or treatment adherence recorded at different time points. These covariates allow for capturing the dynamic and personalized aspects of each unit's experience.

These three options are available for estimation, mirroring the description of the estimator in [Athey et al. (2021)](https://www.tandfonline.com/doi/full/10.1080/01621459.2021.1891924).

In the `generate_data` function, you can control the inclusion of these covariates using the boolean flags X_cov, Z_cov, and V_cov. Setting these flags to True incorporates the respective type of covariates into the generated dataset, while setting them to False excludes them.